# Trusted -> Refined

### Imports and Instances

### Read data from S3

### Group by day by sentiment

### Proporcion para cada sentimiento del total de tweets por dia

### Get change percentage

In [ ]:

# mvv = company_dataframe.select('ticker_symbol').rdd.flatMap(lambda x: x).collect()
mvv = [row['ticker_symbol'] for row in company_dataframe.collect()]
print('mvv:')
print(mvv)
print('Downloading yfinance data:')
prices = yf.download(mvv, start="2014-12-31", end="2020-12-31")['Adj Close']
price=pd.DataFrame(index=pd.date_range(start="2014-12-31",end="2020-12-31"))
price.index.name='Date'
price = pd.concat([price,prices],axis=1)
price=price.fillna(method='ffill')
# print(price.isnull().sum().sum()==0)
# print(all(pd.date_range(start="2015-01-01",end="2020-12-31").isin(price.index)))
change_price=price.pct_change()

print('change_price.index[:10]')
print(change_price.index[:10])

print('change_price.loc[change_price.index==2014-12-31].index:')
print(change_price.loc[change_price.index=='2014-12-31'].index)

## Eliminar periodos fuera del rango de analisis:
change_price.drop(change_price.loc[change_price.index=='2014-12-31'].index, axis=0, inplace=True)

#tweet_dataframe = tweet_dataframe.join(change_price, tweet_dataframe['post_date'] == change_price.index, 'left')


https://stackoverflow.com/q/48234474


Data Preparation Pipeline


In [ ]:

from pyspark.ml.feature import NGram, Tokenizer, StopWordsRemover
from pyspark.ml.feature import HashingTF, IDF, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vector

indexer = StringIndexer(inputCol='category', outputCol='label')
tokenizer = Tokenizer(inputCol="sentence", outputCol="sentence_tokens")
remove_stop_words = StopWordsRemover(inputCol="sentence_tokens", outputCol="filtered")
unigrammer = NGram(n=1, inputCol="filtered", outputCol="tokens") 
hashingTF = HashingTF(inputCol="tokens", outputCol="hashed_tokens")
idf = IDF(inputCol="hashed_tokens", outputCol="tf_idf_tokens")

clean_up = VectorAssembler(inputCols=['tf_idf_tokens'], outputCol='features')

data_prep_pipe = Pipeline(
    stages=[indexer, tokenizer, remove_stop_words, unigrammer, hashingTF, idf, clean_up]
)
transformed = data_prep_pipe.fit(spark_df).transform(spark_df)
clean_data = transformed.select(['label','features'])



Train the model


In [ ]:

from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()
(training,testing) = clean_data.randomSplit([0.7,0.3], seed=12345)
model = nb.fit(training)
test_results = model.transform(testing)


Evaluate Model


In [ ]:

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting label was: {}".format(acc))